# Keras Hyperparameter Tuning
Based on [https://keras.io/guides/keras_tuner/getting_started/](https://keras.io/guides/keras_tuner/getting_started/)

In [28]:
import keras_tuner

## Setup MNIST data
We will use the MNIST dataset to demonstrate the use of the `Tuner`. The dataset contains 60,000 training images and 10,000 test images of hand-written digits. Each image is 28x28 pixels. We will use the last 10,000 images for validation.
Our model will not use Convolutional layers, so we will flatten the images to 784 pixels. This will be done in the `build_model` function with the `Flatten` layer.

In [20]:
from tensorflow import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


## Build the model

We create a function that takes a hyperparameter dictionary as input and returns a compiled model. This function is called repeatedly by the keras tuner to create models with different hyperparameters.

In [21]:
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten())

    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(units=hp.Int(f'units_{i}', 32, 128, 32),
                                     activation=hp.Choice(f'activation', ['relu', 'tanh'])))
    if hp.Boolean('dropout'):
        model.add(keras.layers.Dropout(rate=hp.Float('dropout_rate', 0, 0.5, step=0.1)))
    model.add(keras.layers.Dense(10, activation='softmax'))

    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


build_model(keras_tuner.HyperParameters())


In [39]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    overwrite=True,
    executions_per_trial=3,
    directory='tuner_dir',
    project_name='intro_to_kt')

tuner.search_space_summary()


Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


## Setup Tensorboard
Don't forget to trust the notebook and run the following cell to start Tensorboard.
```{python}
%load_ext tensorboard
%tensorboard --logdir logs
```
Trusting the Jupyter notebook will allow Tensorboard to be displayed inside the notebook itself, because it requires Javascript to be executed.

Or you can open Tensorboard in a browser window by running the following command in a terminal:
```{python}
%load_ext tensorboard
%tensorboard --logdir logs --host localhost --port 8088
```


In [38]:
%load_ext tensorboard
%tensorboard --logdir logs --host localhost --port 8088

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

## Start the search


In [40]:
tuner.search(x_train, y_train,
             epochs=5,
             validation_data=(x_val, y_val),
             callbacks=[keras.callbacks.TensorBoard(log_dir='logs')])


Trial 10 Complete [00h 00m 54s]
val_accuracy: 0.9735666513442993

Best val_accuracy So Far: 0.9735666513442993
Total elapsed time: 00h 08m 16s
INFO:tensorflow:Oracle triggered exit


## Tuner summary

In [43]:
tuner.results_summary(num_trials=3)

Results summary
Results in tuner_dir\intro_to_kt
Showing 3 best trials
Trial summary
Hyperparameters:
num_layers: 1
units_0: 128
activation: tanh
dropout: False
learning_rate: 0.002708529649489954
units_1: 32
units_2: 96
dropout_rate: 0.0
Score: 0.9735666513442993
Trial summary
Hyperparameters:
num_layers: 2
units_0: 128
activation: tanh
dropout: True
learning_rate: 0.0016618360336245936
units_1: 64
units_2: 64
dropout_rate: 0.1
Score: 0.9731666843096415
Trial summary
Hyperparameters:
num_layers: 3
units_0: 32
activation: relu
dropout: True
learning_rate: 0.0010438306464659397
units_1: 96
units_2: 32
dropout_rate: 0.0
Score: 0.967466672261556


## Get the best model hyperparameters

In [41]:
import pandas as pd

pd.DataFrame(tuner.get_best_hyperparameters()[0].values, index=['optimal value'])


,num_layers,units_0,activation,dropout,learning_rate,units_1,units_2,dropout_rate
optimal value,1,128,tanh,False,0.002709,32,96,0.0


In [37]:
# Get the best models
models = tuner.get_best_models()
# select the first model from list
best_model = models[0]
# Build the model. Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(None, 28, 28))
best_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 96)                75360     
                                                                 
 dense_1 (Dense)             (None, 32)                3104      
                                                                 
 dense_2 (Dense)             (None, 10)                330       
                                                                 
Total params: 78,794
Trainable params: 78,794
Non-trainable params: 0
_________________________________________________________________
